In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/kmeans/data_description.txt
/kaggle/input/kmeans/label.csv
/kaggle/input/kmeans/data.csv


In [2]:
data = pd.read_csv("/kaggle/input/kmeans/data.csv")
labels = pd.read_csv("/kaggle/input/kmeans/label.csv")

In [3]:
# Function to compute Euclidean distance
def euclidean_distance(a, b):
#     return np.linalg.norm(a - b)
    return np.sqrt(np.sum((a - b) ** 2))

# Function to compute Cosine similarity
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return 1 - (dot_product / (norm_a * norm_b))

# Function to compute Generalized Jaccard similarity
def generalized_jaccard_similarity(a, b):
    intersection = np.sum(np.minimum(a, b))
    union = np.sum(np.maximum(a, b))
    return 1 - (intersection / union)

# Function to initialize centroids randomly
def initialize_centroids(data, k):
    np.random.seed(42)  # Set seed for reproducibility
    indices = np.random.choice(data.shape[0], k, replace=False)
    return data[indices]

# Function to perform K-means clustering
def kmeans_clustering(data, k, distance_metric, max_iters=100):
    centroids = initialize_centroids(data, k)
    n_samples = data.shape[0]
    n_features = data.shape[1]
    clusters = np.zeros(n_samples)
    
    for _ in tqdm(range(max_iters)):
        for i in range(n_samples):
            distances = [distance_metric(data[i], centroid) for centroid in centroids]
            cluster_assignment = np.argmin(distances)
            clusters[i] = cluster_assignment
        
        prev_centroids = centroids.copy()
        for cluster in range(k):
            cluster_points = [data[i] for i in range(n_samples) if clusters[i] == cluster]
            if len(cluster_points) > 0:
                centroids[cluster] = np.mean(cluster_points, axis=0)
        
        if np.all(prev_centroids == centroids):
            break
    
    return clusters, centroids

# Function to calculate Sum of Squared Errors (SSE)
def calculate_sse(data, clusters, centroids, distance_metric):
    sse = 0
    for i, centroid in enumerate(centroids):
        cluster_points = data[clusters == i]
        for point in cluster_points:
            sse += euclidean_distance(point, centroid) ** 2
    return sse

# Assume 'data' contains your dataset with 784 features

# Run K-means with Euclidean distance
k = 10
euclidean_clusters, euclidean_centroids = kmeans_clustering(data.values, k, euclidean_distance)
euclidean_sse = calculate_sse(data.values, euclidean_clusters, euclidean_centroids, euclidean_distance)
print("SSE of Euclidean K-means:", euclidean_sse)

# Run K-means with Cosine similarity
cosine_clusters, cosine_centroids = kmeans_clustering(data.values, k, cosine_similarity)
cosine_sse = calculate_sse(data.values, cosine_clusters, cosine_centroids, cosine_similarity)
print("SSE of Cosine K-means:", cosine_sse)

# Run K-means with Generalized Jaccard similarity
jaccard_clusters, jaccard_centroids = kmeans_clustering(data.values, k, generalized_jaccard_similarity)
jaccard_sse = calculate_sse(data.values, jaccard_clusters, jaccard_centroids, generalized_jaccard_similarity)
print("SSE of Jaccard K-means:", jaccard_sse)


 59%|█████▉    | 59/100 [02:08<01:28,  2.17s/it]


SSE of Euclidean K-means: 25323469145.0


 34%|███▍      | 34/100 [01:45<03:24,  3.10s/it]


SSE of Cosine K-means: 25634288495.0


 30%|███       | 30/100 [01:16<02:59,  2.57s/it]


SSE of Jaccard K-means: 25517095430.0


In [4]:
from scipy import stats

# Function to label clusters using majority vote
def label_clusters(clusters, true_labels, k):
    cluster_labels = []
    for i in range(k):
        cluster_indices = np.where(clusters == i)[0]  # Get indices of data points in cluster i
        cluster_points = true_labels[cluster_indices]
        mode, _ = stats.mode(cluster_points)
        cluster_labels.append(mode)
    return cluster_labels

# Function to compute accuracy
def compute_accuracy(predicted_labels, true_labels):
    correct = 0
    for pred_label, true_label in zip(predicted_labels, true_labels):
        if pred_label == true_label:
            correct += 1
    return correct / len(true_labels) * 100

# Label clusters for Euclidean-K-means
euclidean_cluster_labels = label_clusters(euclidean_clusters, labels.values.flatten(), k)
euclidean_predicted_labels = [euclidean_cluster_labels[int(cluster)] for cluster in euclidean_clusters]
euclidean_accuracy = compute_accuracy(euclidean_predicted_labels, labels.values.flatten())
print("Accuracy of Euclidean K-means:", euclidean_accuracy)

# Label clusters for Cosine-K-means
cosine_cluster_labels = label_clusters(cosine_clusters, labels.values.flatten(), k)
cosine_predicted_labels = [cosine_cluster_labels[int(cluster)] for cluster in cosine_clusters]
cosine_accuracy = compute_accuracy(cosine_predicted_labels, labels.values.flatten())
print("Accuracy of Cosine K-means:", cosine_accuracy)

# Label clusters for Jaccard-K-means
jaccard_cluster_labels = label_clusters(jaccard_clusters, labels.values.flatten(), k)
jaccard_predicted_labels = [jaccard_cluster_labels[int(cluster)] for cluster in jaccard_clusters]
jaccard_accuracy = compute_accuracy(jaccard_predicted_labels, labels.values.flatten())
print("Accuracy of Jaccard K-means:", jaccard_accuracy)


Accuracy of Euclidean K-means: 60.31603160316031
Accuracy of Cosine K-means: 64.24642464246425
Accuracy of Jaccard K-means: 62.81628162816282


In [5]:
from tqdm import tqdm

# Function to perform K-means clustering with stopping criteria
def kmeans_clustering_stop(data, k, distance_metric, max_iters=100, stop_criteria=500):
    centroids = initialize_centroids(data, k)
    n_samples = data.shape[0]
    n_features = data.shape[1]
    clusters = np.zeros(n_samples)
    sse_values = []  # To store SSE values over iterations
    
    for iteration in tqdm(range(max_iters)):
        prev_centroids = centroids.copy()
        
        # Assign data points to clusters
        for i in range(n_samples):
            distances = [distance_metric(data[i], centroid) for centroid in centroids]
            cluster_assignment = np.argmin(distances)
            clusters[i] = cluster_assignment
        
        # Update centroids
        for cluster in range(k):
            cluster_points = [data[i] for i in range(n_samples) if clusters[i] == cluster]
            if len(cluster_points) > 0:
                centroids[cluster] = np.mean(cluster_points, axis=0)
        
        # Calculate SSE and check convergence criteria
        sse = calculate_sse(data, clusters, centroids, distance_metric)
        sse_values.append(sse)
        
#         print(sse_values)
        
        if (np.all(prev_centroids == centroids)) or (iteration > 2 and sse_values[-2] < sse_values[-1] and iteration > 0) or (iteration >= stop_criteria):
            break
    
    return clusters, centroids, sse_values

# Run K-means with Euclidean distance and stop criteria
euclidean_clusters, euclidean_centroids, euclidean_sse_values = kmeans_clustering_stop(data.values, k, euclidean_distance)

# Run K-means with Cosine similarity and stop criteria
cosine_clusters, cosine_centroids, cosine_sse_values = kmeans_clustering_stop(data.values, k, cosine_similarity)

# Run K-means with Generalized Jaccard similarity and stop criteria
jaccard_clusters, jaccard_centroids, jaccard_sse_values = kmeans_clustering_stop(data.values, k, generalized_jaccard_similarity)

# Determine the number of iterations for convergence
euclidean_iterations = len(euclidean_sse_values)
cosine_iterations = len(cosine_sse_values)
jaccard_iterations = len(jaccard_sse_values)

print("Number of iterations for Euclidean K-means convergence:", euclidean_iterations)
print("Number of iterations for Cosine K-means convergence:", cosine_iterations)
print("Number of iterations for Jaccard K-means convergence:", jaccard_iterations)


 26%|██▌       | 26/100 [01:13<03:28,  2.82s/it]

Number of iterations for Euclidean K-means convergence: 55
Number of iterations for Cosine K-means convergence: 15
Number of iterations for Jaccard K-means convergence: 27


In [6]:
### SEPARATE FUNCTIONS FOR EACH STOP CRITERIA

# Function to perform K-means clustering with stopping criteria
def kmeans_clustering_stopPrevCentroids(data, k, distance_metric, max_iters=100, stop_criteria=500):
    centroids = initialize_centroids(data, k)
    n_samples = data.shape[0]
    n_features = data.shape[1]
    clusters = np.zeros(n_samples)
    sse_values = []  # To store SSE values over iterations
    
    for iteration in tqdm(range(max_iters)):
        prev_centroids = centroids.copy()
        
        # Assign data points to clusters
        for i in range(n_samples):
            distances = [distance_metric(data[i], centroid) for centroid in centroids]
            cluster_assignment = np.argmin(distances)
            clusters[i] = cluster_assignment
        
        # Update centroids
        for cluster in range(k):
            cluster_points = [data[i] for i in range(n_samples) if clusters[i] == cluster]
            if len(cluster_points) > 0:
                centroids[cluster] = np.mean(cluster_points, axis=0)
        
        # Calculate SSE and check convergence criteria
        sse = calculate_sse(data, clusters, centroids, distance_metric)
        sse_values.append(sse)
        
#         print(sse_values)
        
        if (np.all(prev_centroids == centroids)):
            break
    
    return clusters, centroids, sse_values


def kmeans_clustering_stopCriteria(data, k, distance_metric, max_iters=100, stop_criteria=500):
    centroids = initialize_centroids(data, k)
    n_samples = data.shape[0]
    n_features = data.shape[1]
    clusters = np.zeros(n_samples)
    sse_values = []  # To store SSE values over iterations
    
    for iteration in tqdm(range(max_iters)):
        prev_centroids = centroids.copy()
        
        # Assign data points to clusters
        for i in range(n_samples):
            distances = [distance_metric(data[i], centroid) for centroid in centroids]
            cluster_assignment = np.argmin(distances)
            clusters[i] = cluster_assignment
        
        # Update centroids
        for cluster in range(k):
            cluster_points = [data[i] for i in range(n_samples) if clusters[i] == cluster]
            if len(cluster_points) > 0:
                centroids[cluster] = np.mean(cluster_points, axis=0)
        
        # Calculate SSE and check convergence criteria
        sse = calculate_sse(data, clusters, centroids, distance_metric)
        sse_values.append(sse)
        
#         print(sse_values)
        
        if (iteration >= stop_criteria):
            break
    
    return clusters, centroids, sse_values


def kmeans_clustering_stopSSE(data, k, distance_metric, max_iters=100, stop_criteria=500):
    centroids = initialize_centroids(data, k)
    n_samples = data.shape[0]
    n_features = data.shape[1]
    clusters = np.zeros(n_samples)
    sse_values = []  # To store SSE values over iterations
    
    for iteration in tqdm(range(max_iters)):
        prev_centroids = centroids.copy()
        
        # Assign data points to clusters
        for i in range(n_samples):
            distances = [distance_metric(data[i], centroid) for centroid in centroids]
            cluster_assignment = np.argmin(distances)
            clusters[i] = cluster_assignment
        
        # Update centroids
        for cluster in range(k):
            cluster_points = [data[i] for i in range(n_samples) if clusters[i] == cluster]
            if len(cluster_points) > 0:
                centroids[cluster] = np.mean(cluster_points, axis=0)
        
        # Calculate SSE and check convergence criteria
        sse = calculate_sse(data, clusters, centroids, distance_metric)
        sse_values.append(sse)
        
#         print(sse_values)
        
        if (iteration > 2 and sse_values[-2] < sse_values[-1]):
            break
    
    return clusters, centroids, sse_values

In [7]:
euclidean_clusters, euclidean_centroids, euclidean_sse_values = kmeans_clustering_stopPrevCentroids(data.values, k, euclidean_distance)

# Run K-means with Cosine similarity and stop criteria
cosine_clusters, cosine_centroids, cosine_sse_values = kmeans_clustering_stopPrevCentroids(data.values, k, cosine_similarity)

# Run K-means with Generalized Jaccard similarity and stop criteria
jaccard_clusters, jaccard_centroids, jaccard_sse_values = kmeans_clustering_stopPrevCentroids(data.values, k, generalized_jaccard_similarity)

# Determine the number of iterations for convergence
euclidean_iterations = len(euclidean_sse_values)
cosine_iterations = len(cosine_sse_values)
jaccard_iterations = len(jaccard_sse_values)

print("Number of iterations for Euclidean K-means convergence - prev centroids):", euclidean_iterations)
print("Number of iterations for Cosine K-means convergence - prev centroids:", cosine_iterations)
print("Number of iterations for Jaccard K-means convergence - prev centroids:", jaccard_iterations)


 30%|███       | 30/100 [01:23<03:14,  2.77s/it]

Number of iterations for Euclidean K-means convergence - prev centroids): 60
Number of iterations for Cosine K-means convergence - prev centroids: 35
Number of iterations for Jaccard K-means convergence - prev centroids: 31


In [8]:
euclidean_clusters, euclidean_centroids, euclidean_sse_values = kmeans_clustering_stopCriteria(data.values, k, euclidean_distance)

# Run K-means with Cosine similarity and stop criteria
cosine_clusters, cosine_centroids, cosine_sse_values = kmeans_clustering_stopCriteria(data.values, k, cosine_similarity)

# Run K-means with Generalized Jaccard similarity and stop criteria
jaccard_clusters, jaccard_centroids, jaccard_sse_values = kmeans_clustering_stopCriteria(data.values, k, generalized_jaccard_similarity)

# Determine the number of iterations for convergence
euclidean_iterations = len(euclidean_sse_values)
cosine_iterations = len(cosine_sse_values)
jaccard_iterations = len(jaccard_sse_values)

print("Number of iterations for Euclidean K-means convergence - stop criteria):", euclidean_iterations)
print("Number of iterations for Cosine K-means convergence - stop criteria:", cosine_iterations)
print("Number of iterations for Jaccard K-means convergence - stop criteria:", jaccard_iterations)


100%|██████████| 100/100 [04:32<00:00,  2.73s/it]

Number of iterations for Euclidean K-means convergence - stop criteria): 100
Number of iterations for Cosine K-means convergence - stop criteria: 100
Number of iterations for Jaccard K-means convergence - stop criteria: 100


In [9]:
euclidean_clusters, euclidean_centroids, euclidean_sse_values = kmeans_clustering_stopSSE(data.values, k, euclidean_distance)

# Run K-means with Cosine similarity and stop criteria
cosine_clusters, cosine_centroids, cosine_sse_values = kmeans_clustering_stopSSE(data.values, k, cosine_similarity)

# Run K-means with Generalized Jaccard similarity and stop criteria
jaccard_clusters, jaccard_centroids, jaccard_sse_values = kmeans_clustering_stopSSE(data.values, k, generalized_jaccard_similarity)

# Determine the number of iterations for convergence
euclidean_iterations = len(euclidean_sse_values)
cosine_iterations = len(cosine_sse_values)
jaccard_iterations = len(jaccard_sse_values)

print("Number of iterations for Euclidean K-means convergence - stop sse):", euclidean_iterations)
print("Number of iterations for Cosine K-means convergence - stop sse:", cosine_iterations)
print("Number of iterations for Jaccard K-means convergence - stop sse:", jaccard_iterations)

 26%|██▌       | 26/100 [01:13<03:28,  2.82s/it]

Number of iterations for Euclidean K-means convergence - stop sse): 55
Number of iterations for Cosine K-means convergence - stop sse: 15
Number of iterations for Jaccard K-means convergence - stop sse: 27
